In [147]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [148]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [149]:
%%capture pwd
!pwd

In [150]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
yConfig = alt.Axis(
    title="%",
    domain=False,
    ticks=False,
    labelAlign="left",
    labelBaseline="middle",
    labelPadding=-5,
    labelOffset=-10,
    titleX=35,
    titleY=-6,
    titleAngle=0,
    titleFontSize=11,
    titleAlign="left",
    tickCount=5,
    format=".0f",
)
xConfig = alt.Axis(
    grid=False,
    title="",
    titleAnchor="end",
    titleY=-15,
    labelColor=colors["eco-gray"],
    titleColor=colors["eco-gray"],
    tickColor=colors["eco-gray"],
    domainColor=colors["eco-gray"],
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [151]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/RDeconomist/observatory/main/Average%20real%20GDP%201910s%20%26%201920s.csv"
)

In [152]:
f = "fig1_twenties"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,Country,Decade,Average real GDP
0,Japan,1910s average,4.6394
1,Japan,1920s average,2.7787
2,Italy,1910s average,0.0386
3,Italy,1920s average,3.7526
4,France,1910s average,-1.1251


In [153]:
bars = alt.Chart(f1).encode(
    x=alt.X(
        "Average real GDP:Q",
        stack=False,
        title="",
        axis=alt.Axis(
            grid=False,
            title="%",
            titleAnchor="end",
            titleX=415,
            titleY=7,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    y=alt.Y("Country:N", title="", axis=None),
)

bars1 = (
    bars.mark_bar(size=11, yOffset=-10, color=colors["eco-turquiose"], opacity=0.8)
    .transform_filter("datum.Decade=='1910s average'")
    .transform_filter("datum['Average real GDP']>0")
)
labels1 = (
    bars1.mark_text(align="left", xOffset=5, yOffset=-10, color=colors["eco-turquiose"])
    .encode(text="label:N")
    .transform_calculate(label="'+'+round(datum['Average real GDP']*10)/10")
)
bars2 = (
    bars.mark_bar(size=11, yOffset=-10, color=colors["eco-pink"], opacity=0.8)
    .transform_filter("datum.Decade=='1910s average'")
    .transform_filter("datum['Average real GDP']<=0")
)
labels2 = (
    bars2.mark_text(align="right", xOffset=-5, yOffset=-10, color=colors["eco-pink"])
    .encode(text="label:N")
    .transform_calculate(label="round(datum['Average real GDP']*10)/10")
)
labels2b = (
    bars2.mark_text(align="right", xOffset=-3, yOffset=-10, color="#ffffff")
    .encode(text="label:N", x="x:Q")
    .transform_calculate(x="0")
    .transform_calculate(label="'1910s'")
)
bars3 = (
    bars.mark_bar(size=11, yOffset=2, color=colors["eco-light-blue"], opacity=0.8)
    .transform_filter("datum.Decade=='1920s average'")
    .transform_filter("datum['Average real GDP']>0")
)
labels3 = (
    bars3.mark_text(align="left", xOffset=5, yOffset=4, color=colors["eco-light-blue"])
    .encode(text="label:N")
    .transform_calculate(label="'+'+round(datum['Average real GDP']*10)/10")
)
labels1b = (
    bars1.mark_text(align="left", xOffset=3, yOffset=-10, size=10, color="#ffffff")
    .encode(text="label:N", x="x:Q")
    .transform_calculate(x="0")
    .transform_calculate(label="'1910s'")
    .transform_filter("datum['Average real GDP']>1")
)
labels3b = (
    bars3.mark_text(align="left", xOffset=3, yOffset=3, size=10, color="#ffffff")
    .encode(text="label:N", x="x:Q")
    .transform_calculate(x="0")
    .transform_calculate(label="'1920s'")
    .transform_filter("datum['Average real GDP']>1")
    .transform_filter("datum['Average real GDP']<5")
)
labels3b2 = (
    bars3.mark_text(align="left", xOffset=3, yOffset=3, size=10, color="#ffffff")
    .encode(text="label:N", x="x:Q")
    .transform_calculate(x="0")
    .transform_calculate(label="'1920s average'")
    .transform_filter("datum['Average real GDP']>5")
)
labels = (
    bars.mark_text(align="right", xOffset=-5, yOffset=4, color=colors["eco-gray"])
    .encode(text="Country:N", x="x:Q")
    .transform_calculate(x="0")
)
rule = (
    alt.Chart(pd.DataFrame([{"x": 0}]))
    .mark_rule(color=colors["eco-gray"])
    .encode(x="x:Q")
)

layers = (
    (
        bars1
        + labels1
        + bars2
        + labels2
        + bars3
        + labels3
        + rule
        + labels
        + labels2b
        + labels1b
        + labels3b
        + labels3b2
    )
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)

layers.save("visualisation/" + f + ".json")
layers

alt.LayerChart(...)